#### Audio and Speech
- 음성 비서
    - AI 콜센터, 음성 챗봇, 자동차 음성 비서
    - speech-to-speech(Realtime)
- Text-to-Speech (텍스트 사람 목소리로 변환)
- Speech-to-Text (음성을 텍스트로 변환)
- RealTime API(audiio -> audio, text), Transcription API(audio->text), Speech API(text->audio) 사용, chat.completion(audio, text -> audio,text)

- Response API 는 지원 안함

In [1]:
from dotenv import load_dotenv, find_dotenv
# .env 파일 가져오기
load_dotenv(find_dotenv())

True

In [ ]:
import base64
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-audio",
    modalities=["text", "audio"],
    audio={"voice": "alloy", "format": "wav"},
    messages=[
        {
            "role": "user",
            "content": "Is a golden retriever a good family dog?"
        }
    ]
)

print(completion.choices[0])

wav_bytes = base64.b64decode(completion.choices[0].message.audio.data)
with open("./python/audio/dog.wav", "wb") as f:
    f.write(wav_bytes)

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=ChatCompletionAudio(id='audio_69981521f12c8191854007f586b7d395', data='...', expires_at=1771578161, transcript='Yes, a golden retriever is known to be an excellent family dog. They are friendly, patient, and great with children. Their gentle temperament and loyalty make them a popular choice for households. They do need plenty of exercise and attention, but overall, they are loving and adaptable companions.'), function_call=None, tool_calls=None))


In [5]:
# text -> 음성

speech = client.audio.speech.create(
    model="gpt-4o-mini-tts",
    voice="ash",
    input="안녕하세요"
)

with open("./python/audio/tts.wav", "wb") as f:
    f.write(speech.read())

In [6]:
# audio -> text

with open("./python/audio/obama.mp3", "rb") as f:
    transcript = client.audio.transcriptions.create(
        model="gpt-4o-mini-tts",
        file=f
    )

print(transcript.text)

NotFoundError: Error code: 404 - {'error': {'code': None, 'message': 'Invalid URL (POST /v1/audio/transcriptions)', 'param': None, 'type': 'invalid_request_error'}, 'usage': {'type': 'duration', 'seconds': 0}}

In [7]:
import base64
import requests
from openai import OpenAI

client = OpenAI()

# Fetch the audio file and convert it to a base64 encoded string
url = "https://cdn.openai.com/API/docs/audio/alloy.wav"
response = requests.get(url)
response.raise_for_status()
wav_data = response.content
encoded_string = base64.b64encode(wav_data).decode('utf-8')

completion = client.chat.completions.create(
    model="gpt-audio",
    modalities=["text", "audio"],
    audio={"voice": "alloy", "format": "wav"},
    messages=[
        {
            "role": "user",
            "content": [
                { 
                    "type": "text",
                    "text": "What is in this recording?"
                },
                {
                    "type": "input_audio",
                    "input_audio": {
                        "data": encoded_string,
                        "format": "wav"
                    }
                }
            ]
        },
    ]
)

print(completion.choices[0].message)

# audio => question based on audio => answer: text+audio 
# 답변으로 받은 텍스트 출력
print(f"답변 {completion.choices[0].message.audio.transcript}")

# 답변으로 받은 오디오 출력
audio_base64 = completion.choices[0].message.audio.data
audio_bytes = base64.b64decode(audio_base64)

with open("./python/audio/response.wav","wb") as fw:
    fw.write(audio_bytes)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=ChatCompletionAudio(id='audio_69981e2ce53881918e21be82a7a39b1f', data='...', expires_at=1771580476, transcript="This recording is stating that the sun rises in the east and sets in the west, and it mentions that humans have observed this for thousands of years. It's a simple explanation of a fundamental fact about how the sun appears to move across the sky."), function_call=None, tool_calls=None)
답변 This recording is stating that the sun rises in the east and sets in the west, and it mentions that humans have observed this for thousands of years. It's a simple explanation of a fundamental fact about how the sun appears to move across the sky.
